### __Option Pricing on S&P 500 Daily Risk Control 10% Index__

__The S&P 500 Daily Risk Control 10% Index (SPXT10UE) is part of S&P Dow Jones Risk Control Indices familly and use the below parameters__

| ***Index Name*** | ***Underlying Risk Index*** | ***Risk Control Level*** | ***Maximum Leverage*** | ***Interest Rate*** | ***Volatility Calculation*** | ***Return Frequency for Volatility*** | ***Lag to Rebalancing Date*** | ***Decay Factor (Short-Term)*** | ***Decay Factor (Long-Term)*** | ***Rebalancing Frequency*** | ***Launch Date*** | ***Bloomberg Tickers***                                                           |
|------------------------------------------------|------------------------------------------------------|------------------------|----------------------|-------------------------|----------------------------|--------------------------------------|-----------------------------|-------------------------------|------------------------------|---------------------------|---------------|---------------------------------------------------------------------------------|
| S&P 500 Daily Risk Control 10% Index|S&P 500 Total Return: SPTR (USD) | 10% | 150%                 | SOFR + 0.02963*         | Exponentially weighted    | Daily                                | 2 days                      | 94%                           | 97%                          | Daily                     | 10-Sep-09     | ***Excess Return:*** SPXT5UE (USD)|


__The S&P 500 Daily Risk Control indices are computed using the below methodology__

$$
\text{Risk Control ER Index Value}_t 
= 
\text{RiskControlERIndexValue}_{rb}
\,\times\,
\Biggl[
1
\;+\;
K_{rb}\,\biggl(\frac{\text{UnderlyingIndex}_t}{\text{UnderlyingIndex}_{t-1}} \;-\; 1\biggr)
\;-\;
K_{rb}\,\Bigl(
  \prod_{i=rb+1}^{t}\bigl(1 + \text{InterestRate}_{i-1} \times \frac{D_{i-1,i}}{360}\bigr)
  \;-\; 1
\Bigr)
\Biggr]
$$


$$
K_{rb} 
= 
\min\!\Bigl(\text{Max }K,\;\frac{\text{Target Volatility}}{\text{Realized Volatility}_{rb-d}}\Bigr)
$$


$$
\text{RealizedVolatility}_t 
  = \max\bigl(\text{RealizedVolatility}_{S,t}, \text{RealizedVolatility}_{L,t}\bigr)
$$

$$
\text{RealizedVolatility}_{S,t} 
  = \sqrt{\frac{252}{n}\,\text{Variance}_{S,t}}
$$

$$
\text{RealizedVolatility}_{L,t}
  = \sqrt{\frac{252}{n}\,\text{Variance}_{L,t}}
$$

$$
\text{Variance}_{S,t}
  = \lambda_S\,\text{Variance}_{S,t-1}
   + \bigl(1 - \lambda_S\bigr)\,\left[
       \ln\!\Bigl(\frac{\text{UnderlyingIndex}_t}{\text{UnderlyingIndex}_{t-n}}\Bigr)
     \right]^2
$$

$$
\text{Variance}_{L,t}
  = \lambda_L\,\text{Variance}_{L,t-1}
   + \bigl(1 - \lambda_L\bigr)\,\left[
       \ln\!\Bigl(\frac{\text{UnderlyingIndex}_t}{\text{UnderlyingIndex}_{t-n}}\Bigr)
     \right]^2
$$

In [2]:
# Import librairies
import numpy as np
import pandas as pd
import plotly.express as px
from typing import Dict, Optional, Union


In [3]:
def simulate_hhw_paths_with_risk_control(
    # Heston-Hull-White model parameters
    S0: float,
    v0: float,
    r0: float,
    b0: float,
    kappa_v: float,
    theta_v: float,
    sigma_v: float,
    kappa_r: float,
    theta_r: float,
    sigma_r: float,
    rho_sv: float,
    rho_sr: float,
    rho_vr: float,
    T: float,
    N: int,
    M: int,
    seed: int,

    # Risk control parameters
    initial_realized_variance: float,
    target_volatility: float = 0.1,
    lambda_s: float = 0.94,
    lambda_l: float = 0.97,
    max_k: float = 1.5,
    lag: int = 2,
    initial_risk_control_index_value: float = 100.0,
    interest_rate_basis: float = 0.0002963,

    # Optional override for the first Brownian increment at t=1
    spot_scenario: float = None,
) -> Dict[int, Dict[str, np.ndarray]]:
    """
    Simulate Monte Carlo paths under the Heston-Hull-White model with risk control,
    allowing an override of the first Brownian increment dW1 at t=1 while preserving
    the correlation structure with dW2 and dW3.

    If override_dW1 is provided, it will replace the random dW1 at t=1 for all paths.
    """
    dt = T / M

    # Build correlation matrix and its Cholesky factor
    corr = np.array([
        [1.0, rho_sv, rho_sr],
        [rho_sv, 1.0, rho_vr],
        [rho_sr, rho_vr, 1.0]
    ])
    L = np.linalg.cholesky(corr)

    # Allocate arrays
    S = np.zeros((M + 1, N))
    v = np.zeros((M + 1, N))
    r = np.zeros((M + 1, N))
    S[0, :] = S0
    v[0, :] = v0
    r[0, :] = r0

    rng = np.random.default_rng(seed)
    sqrt_dt = np.sqrt(dt)

    for t in range(1, M + 1):
        # Generate independent standard normals
        Z = rng.standard_normal((3, N))

        # Override dW1 at t=1 if requested
        if spot_scenario is not None and t == 1:
            # Broadcast scalar to array if needed
            override_dW1 = ((np.log(1+spot_scenario) - (r0 + b0 - 0.5 * v0) * dt) / np.sqrt(v0))
            if np.isscalar(override_dW1):
                dW1_arr = np.full(N, override_dW1)
            else:
                dW1_arr = np.asarray(override_dW1)
                if dW1_arr.shape != (N,):
                    raise ValueError("override_dW1 must be scalar or shape (N,)")
            # Compute corresponding Z1 values from the override:
            # dW1 = L[0,0] * Z1 * sqrt(dt)  => Z1 = dW1 / (L[0,0] * sqrt(dt))
            Z1_override = dW1_arr / (L[0, 0] * sqrt_dt)
            Z[0, :] = Z1_override

        # Compute correlated increments
        dW = L @ Z * sqrt_dt
        dW1, dW2, dW3 = dW[0], dW[1], dW[2]

        # Full-truncation Euler for variance
        v_prev = np.maximum(v[t-1, :], 0.0)
        v[t, :] = (
            v[t-1, :]
            + kappa_v * (theta_v - v_prev) * dt
            + sigma_v * np.sqrt(v_prev) * dW2
        )
        v[t, :] = np.maximum(v[t, :], 0.0)

        # Hull-White short rate
        r[t, :] = (
            r[t-1, :]
            + kappa_r * (theta_r - r[t-1, :]) * dt
            + sigma_r * dW3
        )

        # Asset price log-Euler
        S[t, :] = (
            S[t-1, :]
            * np.exp((r[t-1, :] + b0 - 0.5 * v_prev) * dt + np.sqrt(v_prev) * dW1)
        )

    # Process each path and compute risk control index
    results = {}
    rc_results = {}
    s_results = {}
    K_results = {}
    for i in range(N):
        # Extract path data
        S_path = S[:, i]
        v_path = v[:, i]
        r_path = r[:, i]
        
        # 1. Compute risky asset returns
        risky_asset_returns = np.zeros(M + 1)
        risky_asset_returns[1:] = (S_path[1:] - S_path[:-1]) / S_path[:-1]
        
        # 2. EWMA variances
        variance_s = np.zeros(M + 1)
        variance_l = np.zeros(M + 1)
        variance_s[0] = initial_realized_variance
        variance_l[0] = initial_realized_variance
        
        for j in range(1, M + 1):
            r2 = risky_asset_returns[j] ** 2 if not np.isnan(risky_asset_returns[j]) else 0.0
            # Short-run variance
            variance_s[j] = (1 - lambda_s) * r2 + lambda_s * variance_s[j - 1]
            # Long-run variance
            variance_l[j] = (1 - lambda_l) * r2 + lambda_l * variance_l[j - 1]
        
        # 3. Annualized volatilities
        volatility_s_annualized = np.sqrt(variance_s * 252)
        volatility_l_annualized = np.sqrt(variance_l * 252)
        
        # 4. Realized volatility (max of the two), lagged and back-filled
        realized_volatility = np.maximum(volatility_s_annualized, volatility_l_annualized)
        
        # Apply lag and back-fill (reproduce pandas shift(lag).bfill() behavior)
        if lag > 0:
            realized_volatility_lagged = np.zeros_like(realized_volatility)
            realized_volatility_lagged[lag:] = realized_volatility[:-lag]
            # Back-fill: fill first 'lag' positions with the first shifted value
            # This reproduces df.shift(lag).bfill() behavior
            realized_volatility_lagged[:lag] = realized_volatility[0]
            realized_volatility = realized_volatility_lagged
        
        # 5. Leverage factor K, capped at max_k
        K = np.minimum(target_volatility / realized_volatility, max_k)
        
        # 6. Interest rate (shifted by 1)
        interest_rate = np.zeros(M + 1)
        interest_rate[1:] = (r_path[:-1] + interest_rate_basis)
        
        # 7. Risk control index returns
        risk_control_index_returns = K * (risky_asset_returns - interest_rate / 252)
        
        # 8. Cumulative index level
        # Handle NaN values by setting them to 0
        risk_control_index_returns = np.where(np.isnan(risk_control_index_returns), 
                                            0.0, risk_control_index_returns)
        
        risk_control_index_value = np.zeros(M + 1)
        risk_control_index_value[0] = initial_risk_control_index_value
        
        for j in range(1, M + 1):
            risk_control_index_value[j] = (risk_control_index_value[j-1] * 
                                         (1 + risk_control_index_returns[j]))
        
        # Store results for this path
        results[i] = {
            'S': S_path,
            'v': v_path,
            'r': r_path,
            'risky_asset_returns': risky_asset_returns,
            'variance_s': variance_s,
            'variance_l': variance_l,
            'volatility_s_annualized': volatility_s_annualized,
            'volatility_l_annualized': volatility_l_annualized,
            'realized_volatility': realized_volatility,
            'K': K,
            'interest_rate': interest_rate,
            'risk_control_index_returns': risk_control_index_returns,
            'risk_control_index_value': risk_control_index_value
        }
        K_results[i] = K
        s_results[i] = S_path
        rc_results[i] = risk_control_index_value
    
    return K_results, s_results, rc_results, results

In [4]:
# Define parameters for the simulation ~ Black-Scholes
params = {

    # Heston-Hull-White model parameters
    'S0': 100.0, 
    'v0': 0.2**2, 
    'r0': 0.03, 
    'b0': 0.01,
    
    # Heston parameters
    'kappa_v': 10.0, 
    'theta_v': 0.2**2, 
    'sigma_v': 0.0001,
    'rho_sv': 0.0, 

    # Hull-White parameters
    'kappa_r': 10.0, 
    'theta_r': 0.03, 
    'sigma_r': 0.0001,
    'rho_sr': 0.0, 

    # X parameter
    'rho_vr': 0.0,

    # Simulation parameters
    'T': 10.0, 
    'N': 10000, 
    'M': 2520, 
    'seed': 42,

    # Risk control parameters
    'initial_realized_variance': 0.2**2 / 252,

    # Scnario parameters
    'spot_scenario' : None,  # No override for the first Brownian increment
}

K_results, s_results, rc_results, results = simulate_hhw_paths_with_risk_control(**params)

df_K = pd.DataFrame(K_results)
df_s = pd.DataFrame(s_results)
df_rc = pd.DataFrame(rc_results)

forward = df_rc.iloc[-1].mean()
atm_call_price = (df_rc.iloc[-1]-100).clip(lower=0).mean()

print(f"Forward: {forward:.4f}, ATM Call Price: {atm_call_price:.4f}")

Forward: 105.3158, ATM Call Price: 15.8611


In [5]:
new_params = params.copy()

# Heston parameters
new_params['kappa_v'] = 2
new_params['theta_v'] = 0.2**2
new_params['sigma_v'] = 0.3
new_params['rho_sv'] = -0.

# Hull-White parameters
new_params['kappa_r'] = 0.3
new_params['theta_r'] = 0.03
new_params['sigma_r'] = 0.02
new_params['rho_sr'] = -0.

# X parameter
new_params['rho_vr'] = 0.

K_results, s_results, rc_results, results = simulate_hhw_paths_with_risk_control(**new_params)

df_K = pd.DataFrame(K_results)
df_s = pd.DataFrame(s_results)
df_rc = pd.DataFrame(rc_results)

forward = df_rc.iloc[-1].mean()
atm_call_price = (df_rc.iloc[-1]-100).clip(lower=0).mean()

print(f"Forward: {forward:.4f}, ATM Call Price: {atm_call_price:.4f}")

Forward: 106.1217, ATM Call Price: 16.2179


In [9]:
new_params = params.copy()

# Heston parameters
new_params['kappa_v'] = 2
new_params['theta_v'] = 0.2**2
new_params['sigma_v'] = 0.3
new_params['rho_sv'] = -0.

# Hull-White parameters
new_params['kappa_r'] = 0.3
new_params['theta_r'] = 0.03
new_params['sigma_r'] = 0.02
new_params['rho_sr'] = 0.90

# X parameter
new_params['rho_vr'] = 0.

K_results, s_results, rc_results, results = simulate_hhw_paths_with_risk_control(**new_params)

df_K = pd.DataFrame(K_results)
df_s = pd.DataFrame(s_results)
df_rc = pd.DataFrame(rc_results)

forward = df_rc.iloc[-1].mean()
atm_call_price = (df_rc.iloc[-1]-100).clip(lower=0).mean()

print(f"Forward: {forward:.4f}, ATM Call Price: {atm_call_price:.4f}")

Forward: 106.1219, ATM Call Price: 16.2194


In [ ]:
new_params = params.copy()

# Heston parameters
new_params['kappa_v'] = 2
new_params['theta_v'] = 0.2**2
new_params['sigma_v'] = 0.3
new_params['rho_sv'] = -0.7

# Hull-White parameters
new_params['kappa_r'] = 0.3
new_params['theta_r'] = 0.03
new_params['sigma_r'] = 0.02
new_params['rho_sr'] = 0.90

# X parameter
new_params['rho_vr'] = -0.5

K_results, s_results, rc_results, results = simulate_hhw_paths_with_risk_control(**new_params)

df_K = pd.DataFrame(K_results)
df_s = pd.DataFrame(s_results)
df_rc = pd.DataFrame(rc_results)

forward = df_rc.iloc[-1].mean()
atm_call_price = (df_rc.iloc[-1]-100).clip(lower=0).mean()

print(f"Forward: {forward:.4f}, ATM Call Price: {atm_call_price:.4f}")